In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv3D, Conv3DTranspose, MaxPooling3D, UpSampling3D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
from sys import getsizeof
np.set_printoptions(threshold=np.inf)
import os 
from keras.optimizers import RMSprop





In [ ]:

def Encoder():
    inp = tf.keras.Input(shape=(32,256,256,1)) # prima era 64

    enc = tf.keras.layers.Conv3D(16, (2,2,2), kernel_initializer='lecun_normal', activation='selu', padding = 'same')(inp)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same')(enc)
 
    enc = tf.keras.layers.Conv3D(32, (2,2,2), kernel_initializer='lecun_normal', activation='selu', padding = 'same')(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same')(enc)
 
    enc = tf.keras.layers.Conv3D(64, (2,2,2), kernel_initializer='lecun_normal', activation='selu', padding = 'same')(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same') (enc)

    enc = tf.keras.layers.Conv3D(32, (2,2,2), kernel_initializer='lecun_normal', activation='selu', padding = 'same')(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same') (enc)

    enc = tf.keras.layers.Conv3D(16, (2,2,2), kernel_initializer='lecun_normal', activation='selu', padding = 'same')(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same') (enc)

    # latentent code vae
    latent_code = tf.keras.layers.Flatten()(enc)
    latent_code = tf.keras.layers.Dense(64, )(latent_code) #act fun removed.. linear # prima era 256,# prima non c'era sigmoid
    
    encoder = tf.keras.Model(inp, latent_code, name = 'encoder')
    return encoder


def Encoder2():
    inp = tf.keras.Input(shape=(32,256,256,1)) # prima era 64

    enc = tf.keras.layers.Conv3D(16, (2,2,2), padding = 'same')(inp)
    enc = tf.keras.layers.LeakyReLU(0.2)(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same')(enc)
 
    enc = tf.keras.layers.Conv3D(32, (2,2,2), padding = 'same')(enc)
    #enc = tf.keras.layers.BatchNormalization()(enc)
    enc = tf.keras.layers.LeakyReLU(0.2)(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same')(enc)
 
    enc = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same')(enc)
    #enc = tf.keras.layers.BatchNormalization()(enc)
    enc = tf.keras.layers.LeakyReLU(0.2)(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same') (enc)

    enc = tf.keras.layers.Conv3D(32, (2,2,2), padding = 'same')(enc)
    #enc = tf.keras.layers.BatchNormalization()(enc)
    enc = tf.keras.layers.LeakyReLU(0.2)(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same') (enc)

    enc = tf.keras.layers.Conv3D(16, (2,2,2), padding = 'same')(enc)
    enc = tf.keras.layers.LeakyReLU(0.2)(enc)
    enc = tf.keras.layers.MaxPooling3D((2,2,2), padding = 'same') (enc)

    # latentent code vae
    latent_code = tf.keras.layers.Flatten()(enc)
    latent_code = tf.keras.layers.Dense(128, activation = 'sigmoid')(latent_code) #act fun removed.. linear # prima era 256
    
    encoder = tf.keras.Model(inp, latent_code, name = 'encoder')
    return encoder


def Decoder():
    z = tf.keras.Input(shape=(32,)) # prima era 10 # prima era 256
    
    # start decoder
    rec = tf.keras.layers.Dense(1024, activation='selu')(z) # ripristino le dimensioni complete
    rec = tf.keras.layers.Reshape((1, 8, 8, 16))(rec) # riprestinate le dimensioni
    
    # decoding
    code = tf.keras.layers.Conv3DTranspose(16,(2,2,2), strides=(1,1,1), kernel_initializer='lecun_normal', activation='selu', padding = 'same')(rec)
    code = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(code)

    dec = tf.keras.layers.Conv3DTranspose(32, (2,2,2), strides=(1,1,1), kernel_initializer='lecun_normal', activation='selu', padding='same')(code)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    dec = tf.keras.layers.Conv3DTranspose(64, (2,2,2), strides=(1,1,1), kernel_initializer='lecun_normal', activation='selu', padding='same')(dec)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    dec = tf.keras.layers.Conv3DTranspose(32, (2,2,2), strides=1, kernel_initializer='lecun_normal', activation='selu', padding='same')(dec)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    dec = tf.keras.layers.Conv3DTranspose(16, (2,2,2), strides=1, kernel_initializer='lecun_normal', activation='selu', padding='same')(dec)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    decoded = tf.keras.layers.Conv3D(1, (3,3,3), activation='sigmoid', padding='same')(dec)
    #model
    decoder = tf.keras.Model(inputs = z, outputs = decoded, name = 'decoder')
    return decoder

def Decoder2():
    z = tf.keras.Input(shape=(128,)) # prima era 10 # prima era 256
    
    # start decoder
    rec = tf.keras.layers.Dense(1024, activation='selu')(z) # ripristino le dimensioni complete
    rec = tf.keras.layers.Reshape((1, 8, 8, 16))(rec) # riprestinate le dimensioni
    
    # decoding
    code = tf.keras.layers.Conv3DTranspose(16,(2,2,2), strides=(1,1,1), activation = 'relu', padding = 'same')(rec)
    code = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(code)

    dec = tf.keras.layers.Conv3DTranspose(32, (2,2,2), strides=(1,1,1), activation='relu', padding='same')(code)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    dec = tf.keras.layers.Conv3DTranspose(64, (2,2,2), strides=(1,1,1), activation='relu', padding='same')(dec)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    dec = tf.keras.layers.Conv3DTranspose(32, (2,2,2), strides=1, activation='relu', padding='same')(dec)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    dec = tf.keras.layers.Conv3DTranspose(16, (2,2,2), strides=1, activation='relu', padding='same')(dec)
    dec = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(dec)
    
    decoded = tf.keras.layers.Conv3D(1, (3,3,3), activation='sigmoid', padding='same')(dec)
    #model
    decoder = tf.keras.Model(inputs = z, outputs = decoded, name = 'decoder')
    return decoder

def Discriminator(z_size = 64):# prima era 256
    encoded = tf.keras.Input(shape=(z_size,))
    
    x = tf.keras.layers.Dense(128)(encoded)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    
    prediction = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=encoded, outputs=prediction)
    return model


class ConvAAE3D(tf.keras.Model):
    def __init__(self, encoder, decoder, disc, **kwargs):
        super(ConvAAE3D, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.discriminator = disc
        self.loss_weight = 1
    
    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        
        # enc-dec train
        with tf.GradientTape() as tape:
            enc = self.encoder(data)
            reconstruction = self.decoder(enc)
            reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(data, reconstruction))#prima era binary crossentropy
            reconstruction_loss *= 256 * 256
            total_loss = reconstruction_loss 
            
        grads = tape.gradient(total_loss, self.encoder.trainable_variables + self.decoder.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.encoder.trainable_variables + self.decoder.trainable_variables))
        
        # disc train
        with tf.GradientTape() as dc_tape:
            real_distribution = tf.random.normal([tf.shape(data)[0], 64], mean=0.0, stddev=1.0) #256 = dimensione spazio latente # prima era 256
            
            encoder_output = self.encoder(data, training=True)

            dc_real = self.discriminator(real_distribution, training=True)
            dc_fake = self.discriminator(encoder_output, training=True)

            # Discriminator Loss
            loss_real = tf.keras.losses.binary_crossentropy(tf.ones_like(dc_real)*0.85, dc_real, from_logits=True) # label smoothing
            loss_fake = tf.keras.losses.binary_crossentropy(tf.zeros_like(dc_fake), dc_fake, from_logits=True)
            dc_loss = self.loss_weight * tf.reduce_mean(loss_fake + loss_real)

        # update gradienti
        dc_grads = dc_tape.gradient(dc_loss, self.discriminator.trainable_variables)
        self.optimizer.apply_gradients(zip(dc_grads, self.discriminator.trainable_variables))
        
        with tf.GradientTape() as gen_tape:
            encoder_output = self.encoder(data, training=True)
            dc_fake = self.discriminator(encoder_output, training=True)

            # Generator loss
            #gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = dc_fake, labels = tf.ones_like(dc_fake)))
            gen_loss = self.loss_weight * tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(dc_fake), dc_fake, from_logits=True))
        
        # update gradienti
        gen_grads = gen_tape.gradient(gen_loss, self.encoder.trainable_variables)
        self.optimizer.apply_gradients(zip(gen_grads, self.encoder.trainable_variables))
            
        # metrics
        mse = tf.reduce_mean(tf.keras.losses.MSE(data, reconstruction))
        mae = tf.reduce_mean(tf.keras.losses.MAE(data, reconstruction))
        return {"rec loss": total_loss,
                "mse": mse, 
                "mae": mae, 
                "disc loss": dc_loss, 
                "gen_loss":gen_loss,}
    
    def test_step(self, data):
        if isinstance(data, tuple):
            data = data[0]

        enc = self.encoder(data)
        reconstruction = self.decoder(enc)
        reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(data, reconstruction))
        reconstruction_loss *= 256 * 256
        total_loss = reconstruction_loss #+ kl_loss
        # metrics
        mse = tf.reduce_mean(tf.keras.losses.MSE(data, reconstruction))
        mae = tf.reduce_mean(tf.keras.losses.MAE(data, reconstruction))
        return {
            "rec loss": total_loss,
            "mse": mse,
            "mae": mae,}
        
        
        
    def call(self, inputs): # forward pass Implementation
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(inputs, reconstruction))
        reconstruction_loss *= 256 * 256

        total_loss = reconstruction_loss #+ kl_loss
        # metrics
        mse = tf.reduce_mean(tf.keras.losses.MSE(data, reconstruction))
        mae = tf.reduce_mean(tf.keras.losses.MAE(data, reconstruction))
        
        self.add_metric(total_loss, name='loss', aggregation='mean')
        self.add_metric(mse, name='mse', aggregation='mean')
        self.add_metric(mae, name='mae', aggregation='mean')
        return reconstruction
        
        

def show(imgs,cols, rows):
    # Show lungmasked
    fig=plt.figure(figsize=(30, 20))
    columns = cols
    rows = rows

    for i in range(1, columns*rows +1):
        img = imgs[i-1,:,:]
        fig.add_subplot(rows, columns, i)
        plt.imshow(img, cmap="gray", vmin = 0, vmax = 1)
        plt.title(i, fontsize = 10)
        plt.axis('off');
    plt.show()
Encoder().summary()
Encoder2().summary()
Decoder().summary()
Decoder2().summary()
Discriminator().summary()

In [ ]:
#### carico tutte le serie e le normalizzo ####
x_train = []
for element in os.listdir('../input/resempled-osic'):
    if element.endswith('npy'):
        x_train.append(np.expand_dims(np.load('../input/resempled-osic/'+element),-1))
        

x_train = np.array(x_train) #-0.025847689005221518 # mean
x_train = np.reshape(x_train, (x_train.shape[0], 32, 256, 256,1))

trainx = x_train [0:150]
valx = x_train[150:]

del x_train

print(trainx.shape)
print(np.max(trainx),np.min(trainx))
print(np.mean(trainx))

In [ ]:
aae = ConvAAE3D(Encoder(), Decoder(), Discriminator())
aae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 3e-4 )) #3e-4
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="rec loss",
    factor=0.5,
    patience=5,
    verbose=1,
    mode="auto")

aae.fit(trainx,
        trainx,
        epochs = 55,
        batch_size = 8,
        callbacks=[reduce_lr],
        #validation_data=(valx,valx),
        verbose = 1)

In [ ]:
# learning rate restart x2 times -- intuition based on ccosine decay with lr - restart
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="rec loss",
    factor=0.5,
    patience=5,
    verbose=1,
    mode="auto")
aae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 3e-4 )) #3e-4
aae.fit(trainx,
        trainx,
        epochs = 65,
        batch_size = 8,
        callbacks=[reduce_lr],
        #validation_data=(valx,valx),
        verbose = 1)

In [ ]:
ris = aae.encoder.predict(np.expand_dims(trainx[22,:,:,:],0))
code = ris
code.shape
img = aae.decoder.predict(code)
img.shape
vol = img[0,:,:,:,:]
vol.shape
show(vol[:,:,:,0],8, 4)
show(trainx[22,:,:,:,0],8, 4)

In [ ]:
aae.encoder.save('./encoderLat32BEST2')
aae.decoder.save('./decoderLat32BEST2')
aae.discriminator.save('./discLat32BEST2')
!zip -r /kaggle/working/convae3Dvedinum4.zip /kaggle/working